In [ ]:
import pandas as pd
import numpy as np 

import os
import pickle 
import seaborn as sns
import nltk

from nltk.corpus import stopwords

from wordcloud import WordCloud, STOPWORDS
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from nltk.tokenize import word_tokenize

import re
import warnings
warnings.filterwarnings("ignore")






In [ ]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')



In [ ]:
train.head()


In [ ]:
test.head()


**Data analysis**

In [ ]:
train['label'].unique()

In [ ]:
train.shape

In [ ]:
train.dtypes

In [ ]:
train.isnull().any()

In [ ]:
sns.countplot(train.label)

In [ ]:
train['News_Length'] = train['Text'].str.len()
print(train['News_length'])


In [ ]:
sns.distplot(train['News_Length']).set_title('News Length Distribution');


Data cleaning:

- removing the special characters
- removing the punctuations 

In [ ]:
train['Text'] = train['Text'].str.lower()
train['Text'] = train['Text'].str.replace("[^a-zA-Z]"," ")  
    

In [ ]:
train.head()

Tokenization

In [ ]:
import nltk
nltk.download('punkt')
word = []

for i in train['Text']:  #loops over the text 
    word.append(word_tokenize(i))''

In [ ]:
#creating a stopwords list 

filter_sent  = [] 

stopwords_mal = [" ", " "]

for i in word:
    l = []
    
    for j in i:
        if j not in stopwords_mal:
            l.append(j)
    fiter_sent.append(' '.join(l))
    
            

In [ ]:
train['Text'] = filter_sent

In [ ]:
train.tail()

**Word Clouds**

In [ ]:
def create_wordcloud(words): # takes input as string
    
    wordcloud = WordCloud(width = 800, height = 500, random_state = 21, max_font_size = 110).generate(words)
    plt.figure(figsize = (10, 7))
    plt.imshow(wordcloud, interpolation = "bilinear")
    plt.axis('off')
    plt.show()
    

In [ ]:
subset = train[train.label == "YOUR LABEL HERE"]
text = subset.Text.values # list of sentences
words  = " ".join(text) # convert list of sentences into a paragraph of sentences
create_wordcloud(words)

In [ ]:
subset = train[train.label == "YOUR LABEL HERE"]
text = subset.Text.values # list of sentences
words  = " ".join(text) # convert list of sentences into a paragraph of sentences
create_wordcloud(words)

In [ ]:
subset = train[train.label == "YOUR LABEL HERE"]
text = subset.Text.values # list of sentences
words  = " ".join(text) # convert list of sentences into a paragraph of sentences
create_wordcloud(words)

In [ ]:
subset = train[train.label == "YOUR LABEL HERE"]
text = subset.Text.values # list of sentences
words  = " ".join(text) # convert list of sentences into a paragraph of sentences
create_wordcloud(words)

**Label Encoding**

In [ ]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()

train['label1'] = label_encoder.fit_transform(train['Label'])

train.head()

Data Spliting

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train['Text'],
                                                    train['Label1'],
                                                    test_size = 0.2,
                                                    random = 42)

**Word embeddings**

In [ ]:
#Parameters 
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = 300

In [ ]:
#TF-IDF Vectorizer
tfidf = TfidfVectorizer(encoding = 'utf-8',
                        ngram_range = ngram_range,
                        stop_words = None, 
                        lowercase = False,
                        min_df = min_df,
                        max_features = max_features,
                        norm = 'l2',
                        sublinear_tf = True)

features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print(features_train.shape)

features_test = tfidf.transform(X_test).toarray()
labels_test = y_test

print(features_test.shape)



**Modeling**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state = 1)
model.fit(features_train, labels_train)
model_predictions = model.predict(features_test)
print('Accuracy:',accuracy_score(labels_tests, model_predictions))
print(classification_report(labels_test, model_predictions))


In [ ]:
model.get_params()

In [ ]:
print('Accuracy: ', accuracy_score(labels_test, model_predictions))
pd.crosstab(labels_test, model_predictions)

In [ ]:
#Another Approach
from sklearn.metrics import accuracy_score, cohen_kappa_score, confusion_matrix

modelx = RandomForestClassifier(n_estimators = 300, 
                                max_depth = 150,
                                n_jobs = 1)

modelx.fit(features_train, labels_train)

y_pred = modelx.predict(features_test)

c_mat = confusion_matrix(y_test,y_pred)
kappa = cohen_kappa_score(y_test,y_pred)

print("Confusion Matrix: ",c_mat)
print("Kappa: ",kappa)
print("Accuracy:", acc)

                               

**Operations on imbalanced dataset**

In [ ]:
!pip install imblearn


In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()


In [ ]:
features_train_smote, labels_train_smote = smote.fit_sample(features_train.astype("float"), labels_train)


In [ ]:
from collections import Counter
print('Before SMOTE: ', Counter(labels_train))
print('After SMOTE: ', Counter(labels_train_smote))


In [ ]:
model = RandomForestClassifier(random_state = 1)
model.fit(features_train_smote, labels_train_smote)
y_predict = model.predict(features_test)

print(accuracy_score(labels_test, y_predict))
print(classification_report(labels_test, y_predict))


In [ ]:
print('Accuracy: ', accuracy_score(labels_test, y_predict))
pd.crosstab(labels_test, y_predict)

**Hyper Parameter Tuning(Random Forest)**

In [ ]:
from sklearn.model_selection import GridSearchCV
n_estimators = [100, 300, 500, 800, 1200]
max_depth = [5, 8, 15, 25, 30]
min_samples_split = [2, 5, 10, 15, 100]
min_samples_leaf = [1, 2, 5, 10]

hyperF = dict(n_estimators = n_estimators, 
              max_depth = max_depth, 
              min_samples_split = min_samples_split,
              min_samples_leaf = min_samples_leaf)

gridF = GridSearchCV(model, hyperF, cv = 3, verbose = 1, n_jobs = -1)

bestF = gridF.fit(features_train, labels_train)


In [ ]:
#applying the hyperparameters

from sklearn.ensemble import RandomForestClassifier

model1 = RandomForestClassifier(random_state = 1, 
                                max_depth = 30, 
                                min_samples_leaf = 1,
                                min_samples_split = 2,
                                n_estimators = 300)

model1.fit(features_train, labels_train)

model_predictions = model1.predict(features_test)

print('Accuracy: ', accuracy_score(labels_test, model_predictions))

print(classification_report(labels_test, model_predictions))



KNeighbors Classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier()
model.fit(features_train, labels_train)
model_predictions = model.predict(features_test)
print('Accuracy: ', accuracy_score(labels_test, model_predictions))
print(classification_report(labels_test, model_predictions))


In [ ]:
model.get_params()

In [ ]:
print('Accuracy: ',accuracy_score(labels_test, model_predictions))
pd.crosstab(labels_test, model_predictions)

After balancing the dataset

In [ ]:
model = KNeighborsClassfier()
model.fit(features_train_smote, labels_train_smote)
y_predict = model.predict(features_test)
print(accuracy_score(labels_test, y_predict))
print(classification_report(labels_test, y_predict))


In [ ]:
print('Accuracy: ', accuracy_score(labels_test,y_predict))
pd.crosstab(labels_test, y_predict)


Hyper paramter tuning

In [ ]:
from sklearn.model_selection import GridSearchCV
params_KNN = {'n_neighbors':[1,2,3,4,5,6,7], 'p':[1,2,5]}

gridF = GridSearchCV(model, params_KNN, cv = 5, verbose = 1, n_jobs = -1)

bestF = gridF.fit(features_train, labels_train)

In [ ]:
bestF.best_params_

In [ ]:
#applying the best parameter

model = KNeighborsClassifier(n_neighbors = 7, p = 2)
model.fit(features_train, labels_train)
model_predictions = model.predict(feature_test)

print('Accuracy: ', accuracy_score(labels_test, model_predictions))
print(classification_report(labels_test, model_predictions))


Multinomial Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(features_train, labels_train)
model_predicitions = model.predict(features_test)
print('Accuracy: ', accuracy_score(labels_test, model_predictions))
print(classification_report(labels_test, model_predictions))

In [ ]:
model.get_params()

**Hyper Parametr tuning (multinomial naive bayes)**

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {
    'alpha':[1, 1e-1, 1e-2]
    
}

gridF = GridSearchCV(model, parameters, cv = 10, verbose = 1, n_jobs = -1)

bestF = gridF.fit(features_train, labels_train)

In [ ]:
bestF.best_params_

**BOW + Xgboost Model**

In [ ]:
from sklearn.feature_extraction.text import TfidVectorizer, CountVectorizer
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import scipy
from sklearn.metrics import log_loss
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score


In [ ]:
count_vect = CountVectorizer(analyzer = 'word', token_pattern = r'\w{1,}')
count_vect.fit(train['Text'])

train1_trans = count_vect.transform(train['Text'].values)
labels = train['Label1'].values

X = scipy.sparch.hstack([trian1_trans])
y = labels

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2, random_state = 42)

xgb_model = xgb.XGBClassifier(max_depth = 50,
                              n_estimators = 80,
                              learning_rate = 0.1,
                              colsample_bytree = 0.7,
                              gamma = 0,
                              reg_alpha = 4,
                              objective = 'binary:logistic',
                              eta = 0.3, 
                              silent = 1, 
                              subsample = 0.8).fit(X_train,y_train)

                             
xgb_prediction = xgb_model.predict(X_valid)

In [ ]:
from sklearn.metrics import f1_score, classification_report, accuracy_score

print('training score:' f1_score(y_train, xgb_model.predict(X_train), average = 'macro'))
print('validation score:' f1_score(y_valid, x_model.predict(X_valid),average = 'macro'))
print(classification_report(y_valid, xgb_prediction))


In [ ]:
print('Accuracy: ', accuracy_score(y_valid, xgb_prediction))
pd.crosstab(y_valid, xgb_prediction)



After balancing the dataset

In [ ]:
model = xgb_model
model.fit(features_train_smote, labels_train_smote)
y_predict = model.predict(features_test)
print('Accuracy:', accuracy_score(labels_test, y_predict))
pd.crosstab(labels_test,y_predict)


**Word level TF-IDF**

In [ ]:
tfidf_vect = TfidfVectorizer(analyzer = 'word', ngram_range = (2,3), token_pattern = r'\w{1,}', max_features = 5000)

tfidf_vect.fit(train['Text'])
train_trans1 = tfidf_vect.transform(train['Text'].values)
labels = train['label1'].values
X = scipy.sparse.hstack([train_trans1])
y = labels
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2, random_state = 42)


In [ ]:
from sklearn.metrics import f1_score, classification_report, accuracy_score

xgb_model = xgb.XGBClassifier(max_depth = 50, 
                             n_estimators = 80,
                             learning_rate = 0.1,
                             colsample_bytree = 0.7,
                             gamma = 0, 
                             reg_alpha = 4, 
                             objective = 'binary:logistic', 
                             eta = 0.3, 
                             silent = 1, 
                             sample = 0.8).fit(X_train,y_train)

xgb_predictions =xgb_model.predict(X_valid)

print('word level tf-idf training score:', f1_score(y_train, xgb_model.predict(X_train), average = 'macro'))
print('word level tf-idf validation score: ', f1_score(y_valid, xgb_model.predict(X_train),average = 'macro'))
print(classification_report(y_valid, xgb_prediction))


**N-gram level TF-IDF**

In [ ]:
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', 
                                   token_pattern=r'\w{1,}',
                                   ngram_range=(2,3), 
                                   max_features=5000)

tfidf_vect_ngram.fit(train['Text'])
train_trans1 = tfidf_vect_ngram.transform(train['Text'].values)
labels = train['Label1'].values
X = scipy.sparse.hstack([train_trans1])
y = labels
X_train,X_valid,y_train,y_valid = train_test_split(X,y, test_size = 0.2, random_state = 42)

In [ ]:
xgb_model = xgb.XGBClassifier(max_depth = 50,
                              n_estimators = 80,
                              learning_rate = 0.1,
                              colsample_bytree = 0.7, 
                              gamma = 0, 
                              reg_alpha = 4,
                              objective = 'binary:logistic',
                              eta = 0.3, 
                              silent = 1, 
                              subsample = 0.8).fit(X_train, y_train) 
xgb_prediction = xgb_model.predict(X_valid)

print('n-gram level tf-idf training score:', f1_score(y_train, xgb_model.predict(X_train), average='macro'))
print('n-gram level tf-idf validation score:', f1_score(y_valid, xgb_model.predict(X_valid), average='macro'))
print(classification_report(y_valid, xgb_prediction))

**Character level TF-IDF**

In [ ]:
from sklearn.metrics import f1_score, classification_report, accuracy_score
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char',
                                         token_pattern=r'\w{1,}',
                                         ngram_range=(2,3), 
                                         max_features=5000)

tfidf_vect_ngram_chars.fit(train['Text'])
train_trans1 = tfidf_vect_ngram_chars.transform(train['Text'].values)
labels = train['Label1'].values

X = scipy.sparse.hstack([train_trans1])
y = labels

X_train,X_valid,y_train,y_valid = train_test_split(X,y, test_size = 0.2, random_state = 42)
xgb_model = xgb.XGBClassifier(max_depth=50,
                              n_estimators =80,
                              learning_rate =0.1,
                              colsample_bytree= 0.7, 
                              gamma=0, reg_alpha=4, 
                              objective='binary:logistic', 
                              eta=0.3, silent=1,
                              subsample=0.8).fit(X_train, y_train) 
xgb_prediction = xgb_model.predict(X_valid)

print('character level tf-idf training score:', f1_score(y_train, xgb_model.predict(X_train), average='macro'))
print('character level tf-idf validation score:', f1_score(y_valid, xgb_model.predict(X_valid), average='macro'))
print(classification_report(y_valid, xgb_prediction))